In [50]:
import pandas as pd
import numpy as np
import cv2
import os

DATSTETS = ['breakhis', 'oxford_pet','multi_cancer', 'breast_cancer', 'breakhis_small', 'multi_cancer_small']
MODELS = ['conch', 'resnet', 'uni', 'vit']
METHODS = ['grad-cam', 'score-cam', 'eigen-cam']

def load_and_merge_csvs(folder_path):
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in all_files]
    
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.drop_duplicates(inplace=True)
    
    return merged_df

def get_iou_iou(original_img_path, pred_mask_path):
    pred_mask = cv2.imread(pred_mask_path)
    pred_mask = cv2.cvtColor(pred_mask, cv2.COLOR_RGB2GRAY)
    pred_mask = (pred_mask > 0).astype(np.uint8)

    mask_path = original_img_path.replace("Images", "Masks").replace("_ccd", "")
    mask = cv2.imread(mask_path)
    mask[mask == 255] = 1
    mask = mask[:, :, 0]
    mask = cv2.resize(mask, (pred_mask.shape[1], pred_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

    intersection = np.logical_and(mask, pred_mask)
    union = np.logical_or(mask, pred_mask)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def evaluate_iou(df):
    # df_filtered = df[(df["model"] == model) & (df["num_unfreeze_layer"] == num_unfreezed_layer) & (df["method"] == method)]
    iou_scores = []
    for i in range(len(df)):
        mask_path = df.iloc[i]["original_img_path"]
        pred_mask = df.iloc[i]["mask_path"]
        iou_score = get_iou_iou(mask_path, pred_mask)
        iou_scores.append(iou_score)
    return iou_scores


df = load_and_merge_csvs("./assets/explanations_v2/")
df = df[df["method"].isin(METHODS) & df["dataset"].isin(DATSTETS)]
df.reset_index(drop=True, inplace=True)

In [51]:
iou_df = df[df["dataset"] == "breast_cancer"]
iou_df_groups = iou_df.groupby(["model", "num_unfreeze_layers", "method"])

data = []
for group in iou_df_groups.groups:
    model, num_unfreezed_layer, method = group
    iou_scores = evaluate_iou(iou_df_groups.get_group(group))
    data.append([model, num_unfreezed_layer, method, np.mean(iou_scores), np.std(iou_scores)])

iou_scores_df = pd.DataFrame(data, columns=["model", "num_unfreeze_layers", "method", "mean_iou", "std_iou"])
iou_scores_df.sort_values(by="mean_iou", ascending=False, inplace=True)
#iou_scores_df.to_csv("./assets/iou_scores.csv", index=False)
iou_scores_df.reset_index(drop=True, inplace=True)
iou_scores_df
    

,model,num_unfreeze_layers,method,mean_iou,std_iou
0,uni,4,grad-cam,0.042812,0.039101
1,vit,0,score-cam,0.027563,0.019495
2,uni,0,score-cam,0.026109,0.031912
3,resnet,0,grad-cam,0.022320,0.032771
4,uni,0,grad-cam,0.019124,0.021060
5,conch,0,grad-cam,0.016361,0.014455
6,conch,4,grad-cam,0.012362,0.013292
7,vit,4,grad-cam,0.011785,0.012201
8,vit,4,score-cam,0.011581,0.012289
9,resnet,0,score-cam,0.010059,0.017422


In [52]:
df["road_avg"] = df.apply(lambda x: (x.loc["road_least"] - x.loc["road_most"])/2, axis=1)
road_df_groups = df.groupby(["model", "num_unfreeze_layers", "method"])[["road_most", "road_least", "road_avg"]].mean()
road_df_groups.reset_index(inplace=True)
road_df_groups.sort_values(by=["model", "num_unfreeze_layers", "method"], inplace=True)
# indices = road_df_groups.groupby(["model", "num_unfreeze_layers"])["road_avg"].idxmax()
# road_df_groups = road_df_groups.loc[indices]
# road_df_groups.sort_values(by="road_avg", ascending=False, inplace=True)
# road_df_groups.reset_index(drop=True, inplace=True)
# road_df_groups

road_df_groups = road_df_groups[['model', 'num_unfreeze_layers', 'method', 'road_avg']]
road_df_groups = road_df_groups.sort_values(by=['model', 'num_unfreeze_layers', 'method'])
road_df_groups

,model,num_unfreeze_layers,method,road_avg
0,conch,0,eigen-cam,0.024992
1,conch,0,grad-cam,0.019060
2,conch,0,score-cam,0.041494
3,conch,4,eigen-cam,0.027477
4,conch,4,grad-cam,0.018059
5,conch,4,score-cam,0.054857
6,resnet,0,eigen-cam,-0.013168
7,resnet,0,grad-cam,-0.032766
8,resnet,0,score-cam,0.015100
9,uni,0,eigen-cam,0.019818


In [53]:
road_df_groups_datasets = df.groupby(["model", "num_unfreeze_layers", "method", "dataset"])[["road_most", "road_least", "road_avg"]].mean()
road_df_groups_datasets = road_df_groups_datasets.groupby(["model", "num_unfreeze_layers", "method"])

In [54]:
road_df_groups_datasets.get_group(("conch", 4, "score-cam"))

road_most  road_least  \
model num_unfreeze_layers method    dataset                                     
conch 4                   score-cam breakhis            -0.498893   -0.162636   
                                    breakhis_small      -0.271571   -0.097159   
                                    breast_cancer       -0.010715   -0.010611   
                                    multi_cancer        -0.358238   -0.330973   
                                    multi_cancer_small  -0.583156   -0.581028   
                                    oxford_pet          -0.351712   -0.138572   

                                                        road_avg  
model num_unfreeze_layers method    dataset                       
conch 4                   score-cam breakhis            0.168128  
                                    breakhis_small      0.087206  
                                    breast_cancer       0.000052  
                                    multi_cancer        0.013632  
                                    multi_cancer_small  0.001064  
                                    oxford_pet          0.106570

In [55]:
road_df_groups_datasets.get_group(("uni", 0, "score-cam"))

road_most  road_least  \
model num_unfreeze_layers method    dataset                                     
uni   0                   score-cam breakhis            -0.492204   -0.350223   
                                    breakhis_small      -0.456423   -0.337430   
                                    breast_cancer       -0.042127   -0.045557   
                                    multi_cancer        -0.666335   -0.665161   
                                    multi_cancer_small  -0.664677   -0.639487   
                                    oxford_pet          -0.258843   -0.010298   

                                                        road_avg  
model num_unfreeze_layers method    dataset                       
uni   0                   score-cam breakhis            0.070991  
                                    breakhis_small      0.059496  
                                    breast_cancer      -0.001715  
                                    multi_cancer        0.000587  
                                    multi_cancer_small  0.012595  
                                    oxford_pet          0.124273

In [56]:
# df["road_avg"] = df.apply(lambda x: (x.loc["road_least"] - x.loc["road_most"])/2, axis=1)
# road_df_groups = df.groupby(["model", "num_unfreeze_layers", "method"])

# data = []

# for group in road_df_groups.groups:
#     model, num_unfreezed_layer, method = group
#     df_group = road_df_groups.get_group(group)

#     road_dict = {}

#     for row in df_group.iterrows():
#         row = row[1]
#         road_dict[row["dataset"]] = row["road_avg"]
    
#     road_avg = np.mean(list(road_dict.values()))

#     data.append([model, num_unfreezed_layer, method, road_avg, road_dict])

# road_avg_df = pd.DataFrame(data, columns=["model", "num_unfreeze_layers", "method", "road_avg", "road_dict"])
# road_avg_df.sort_values(by=["model", "num_unfreeze_layers", "method"], inplace=True)
# road_avg_df.reset_index(drop=True, inplace=True)
# # road_avg_df.to_csv("./assets/road_avg.csv", index=False)

# road_avg_df

In [57]:
# road_avg_df = road_avg_df.sort_values(by="road_avg", ascending=False)
# road_avg_df.reset_index(drop=True, inplace=True)
# road_avg_df